<a href="https://colab.research.google.com/github/babybeelz20/Big-Data-Midterm/blob/main/Spark_File_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=5562df2b7df2911d5d6def3c92dda55df173bf6d43432af0c6d720b94352e07b
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


## Initiate the Spark session

In [2]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
# Check spark session
print(spark)

In [4]:
spark.sparkContext.setLogLevel("ERROR")

## Define the data structure

In [8]:
from pyspark.sql.types import StructType,StructField, StringType, DateType

schema = StructType([\
        StructField("link", StringType(), True),\
        StructField("headline", StringType(), True),\
        StructField("category", StringType(), True),\
        StructField("short_description", StringType(), True),\
        StructField("authors", StringType(), True),\
        StructField("date", DateType(), True),\
])

In [13]:
# To allow automatic schemaInference while reading
spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
streaming_df = spark.readStream\
    .format("json") \
    .option("cleanSource", "archive") \
    .option("sourceArchiveDir", "data/archive/device_data/") \
    .option("maxFilesPerTrigger", 1) \
    .load("/content/input-05111940000048")

In [ ]:
# To the schema of the data, place a sample json file and change readStream to read 
streaming_df.printSchema()
streaming_df.show(truncate=False)

In [18]:
# Lets explode the data as devices contains list/array of device reading
from pyspark.sql.functions import explode, col

exploded_df = streaming_df \
    .select("link", "headline", "category", "short_description", "authors", "date") \
    .withColumn("devices", explode("data.devices")) \
    .drop("date")

AnalysisException: ignored

## Determine the folder location

In [9]:
df = spark.readStream.schema(schema).json("/content/input-05111940000048")

In [10]:
df.printSchema()

root
 |-- link: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- category: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- date: date (nullable = true)



## Do the aggregation step

In [11]:
groupDF = df.select("link").groupBy("link").count()

groupDF.printSchema()

root
 |-- link: string (nullable = true)
 |-- count: long (nullable = false)



## Start the streaming task

In [15]:
result = groupDF.writeStream\
  .format("console")\
  .outputMode("complete")\
  .start()


result.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored